# Patient Data

In [ ]:
import pandas as pd
import os
import json
import csv

# Define file paths
csv_file_path = '../data/Demographics.csv'
output_dir = './Output/jsons/'
output_file_path = os.path.join(output_dir, 'Demographics.json')

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Read the CSV file
df = pd.read_csv(csv_file_path)

# Convert the DataFrame to a JSON object
data_json = df.to_dict(orient='records')

# Write the JSON object to a file
with open(output_file_path, 'w') as json_file:
    json.dump(data_json, json_file, indent=4)

print(f"Data has been successfully written to {output_file_path}")

Data has been successfully written to ./Output/jsons/Demographics.json


# Glucose Data

In [13]:
# Define the input file and output directory
data_dir = "../data/"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Initialize the dictionary to store the data
data_dict = {}

#traverse teh data directory to find the Dexcom CSV file
input_file_dir = {}
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.startswith("Dexcom"):
            # get name of directory file is in
            dir_name = os.path.basename(root)

            input_file_dir[dir_name] = os.path.join(root, file)
            break  # Stop after finding the first Dexcom file in directory

# read each Dexcom file and append the data to the dictionary
for key, value in input_file_dir.items():
    # Load the full CSV
    df = pd.read_csv(value)

    # Filter for glucose events (Event Type == 'EGV')
    df_egv = df[df['Event Type'] == 'EGV'].copy()

    # Convert timestamp to datetime format
    df_egv['Timestamp (YYYY-MM-DDThh:mm:ss)'] = pd.to_datetime(df_egv['Timestamp (YYYY-MM-DDThh:mm:ss)'])

    # Rename columns for clarity
    df_egv.rename(columns={
        'Timestamp (YYYY-MM-DDThh:mm:ss)': 'timestamp',
        'Glucose Value (mg/dL)': 'glucose',
    }, inplace=True)

    # Keep only relevant columns
    df_egv = df_egv[['timestamp', 'glucose']].reset_index(drop=True)

    # Convert timestamp to ISO format
    df_egv['timestamp'] = df_egv['timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S')

    # Convert Datafrsame to dictionary
    data_dict[key] = df_egv.to_dict(orient='records')

# Write the data to a JSON file
output_file_path = os.path.join(output_dir, 'glucose.json')
with open(output_file_path, 'w') as json_file:
    json.dump(data_dict, json_file, indent=4)

# Prelim Food_logs

In [34]:
# Define the input file and output directory
data_dir = "../data/"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Initialize the list to store the data
data_list = []

#traverse teh data directory to find the Dexcom CSV file
input_file_dir = {}
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.startswith("Food_Log"):
            # get name of directory file is in
            dir_name = os.path.basename(root)

            input_file_dir[dir_name] = os.path.join(root, file)
            break  # Stop after finding the first Dexcom file in directory


# read each Dexcom file and append the data to the dictionary
for key, value in input_file_dir.items():

    if key == "003":
        # Skip the first file
        continue

    print(key)
    # Load the CSV file
    df_food_log = pd.read_csv(value)

    # Combine 'date' and 'time' into a single datetime column
    df_food_log['datetime'] = pd.to_datetime(df_food_log['date'] + ' ' + df_food_log['time'])

    # Reorder columns
    cols = ['datetime'] + [col for col in df_food_log.columns if col != 'datetime']
    df_food_log = df_food_log[cols]

    # Convert nutrition columns to numeric
    numeric_cols = ['calorie', 'total_carb', 'dietary_fiber', 'sugar', 'protein', 'total_fat']
    df_food_log[numeric_cols] = df_food_log[numeric_cols].apply(pd.to_numeric, errors='coerce')

    # add coloumn for patiendent ID
    df_food_log['ID'] = key

    #add empty columns for tags
    df_food_log['tags'] = ""


    # Convert timestamp to ISO format
    df_food_log['datetime'] = df_food_log['datetime'].dt.strftime('%Y-%m-%dT%H:%M:%S')

    # Convert Datafrsame to dictionary
    data_list.extend(df_food_log.to_dict(orient='records'))

#save the data to a JSON file
output_file_path = os.path.join(output_dir, 'food_log.json')
with open(output_file_path, 'w') as json_file:
    json.dump(data_list, json_file, indent=4)


001
002
004
005
006
007
008
009
010
011
012
013
014
015
016


In [ ]:
import json
import os

output_dir = './Output/jsons/'

input_file_path = os.path.join(output_dir, 'food_log.json')
output_file_path = os.path.join(output_dir, 'food_log_tagged.json')

# Load your JSON data (replace with your file path or variable)
with open(input_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Tag definitions
tag_keywords = {
    'meat': [
        'chicken', 'beef', 'pork', 'turkey', 'sausage', 'bacon', 'ham', 'rib', 'steak', 'chorizo', 'nugget', 'slider', 'wings', 'jerky',
        'pepperoni', 'lamb', 'salami', 'duck', 'lamb gyro', 'duck burrito', 'duck soup with dumpling'
    ],
    'seafood': [
        'anchovies', 'fish', 'salmon', 'shrimp', 'crab', 'tuna', 'seafood', 'sardines', 'squid', 'octopus', 'shellfish', 'lobster', 'scallops',
        'tilapia', 'sushi', 'california roll'
    ],
    'drink': [
        'coca cola', 'beer', 'lemonade', 'boost', 'water', 'soda', 'coffee', 'tea', 'juice', 'milk', 'smoothie', 'shake', 'gatorade', 'powerade',
        'chocolate milk', 'hot chocolate', 'chai', 'sweet tea', 'mello yello', 'mountain dew', 'frozen pop', 'premier protein', 'protein shake',
        'moscato', 'corona', 'mojito', 'chardonnay wine', 'diet coke', 'diet pepsi', 'coke', 'vodka', 'red wine', 'latte', 'decaf latte', 'skim decaf latte',
        'jim beam bourbon', 'gray goose', 'grey goose', 'pepsi', 'diet pepsi', 'diet dr pepper', 'dr pepper', 'sweet tea', 'sweet tea vodka',
        'sweet tea with lemonade', 'sweet tea with vodka', 'sweet tea with sweet tea vodka', 'sweet tea with sweet tea and lemonade', 'sweet tea with sweet tea and vodka',
        'whey protein', 'v8', 'v8 juice', 'v8 splash', 'v8 energy', 'v8 energy drink', 'v8 energy juice', 'v8 energy smoothie', 'v8 energy shake',
    ],
    'entree': [
        'sloppy joe', 'burritos', 'salad', 'wrap', 'bowl', 'sandwich', 'burger', 'pizza', 'mac and cheese', 'ziti', 'sub', 'taco', 'baked chicken',
        'chicken and rice', 'chicken breast', 'chicken thigh', 'chicken wing', 'chicken leg', 'chicken biscuit', 'chicken chorizo', 'chicken nuggets',
        'omelet', 'egg salad', 'baked potato', 'salisbury steak', 'pot pie', 'ravioli', 'taco salad', 'chipotle', 'deluxe cheeseburger macaroni',
        'cheeseburger', 'roast beef', 'baked cheetos', 'totinos pizza', 'lean cuisine', 'lasagna', 'vegetable lasagna', 'stuffing', 'brunswick stew', 'soup'
    ],
    'snack': [
        'fries', 'chips', 'chip', 'trail mix', 'popcorn', 'cookie', 'cookies', 'crackers', 'granola', 'bar', 'jerky', 'muffin', 'nutrigrain', 'fritos',
        'chex mix', 'tootsie', 'oreo', 'fig newton', 'fig bar', 'fruit bar', 'biscotti', 'biscott', 'snickers', 'baby ruth', 'ritz', 'cheetos', 'pistachios',
        'multigrain', 'blue bunny', 'fruit bars', 'string beans', 'grits', 'grapes', 'mandarin', 'tangerine', 'banana bread', 'salsa', 'hummus', 'peanut butter',
        'peppers', 'onions', 'spinach', 'asparagus', 'cabbage', 'rice', 'beans', 'black beans', 'green bean', 'green beans', 'squash', 'corn flakes', 'cornflakes',
        'frosted flakes', 'cereal', 'pita', 'pita bread', 'babybel', 'cheese stick', 'cheese', 'creamers', 'creamer', 'sugar', 'm&ms', 'fruit smoothie',
        'pb protein whey powder', 'blueberries', 'strawberries', 'lemon loaf', 'lemon risotto', 'apple', 'peach', 'orange', 'navel orange', 'townhouse cracker',
        'mini croissants', 'pecan twirl', 'donut', 'hershey kiss', 'pretzel rod', 'mixed nuts', 'cashew nut', 'walnut', 'pecans', 'almonds', 'toffee', 'lifesavers',
        'lifesaver', 'andes creme de menthe', 'sathers caramel creams', 'goetzes caramel cremes', 'peanut m & m', 'm&m\'s', 'sweetner', 'equal', 'stevia', 'honey',
        'raisins', 'gin soaked raisins', 'angel food candy', 'frosted flake', 'sweet potato', 'small sweet potato', 'baked sweet potato', 'thin mints', 'thin mint',
        'mozarrella sticks', 'grape', 'tatziki', 'candy', 'kashi', 'peanut', 'clementine', 'clementines', 'pear', 'grapefruit', 'grapefruits', 'kiwi',
        'kiwis', 'fruit', 'berries', 'berry', 'fruit salad', 'fruit cup', 'fruit cocktail', 'fruit snacks', 'fruit roll up', 'fruit leather', 'fruit smoothie',
    ],
    'dessert': [
        'cake', 'ice cream', 'oreo', 'fig newton', 'fig bar', 'fruit bar', 'biscotti', 'biscott', 'snickers', 'baby ruth', 'ritz', 'cheesecake', 'blue bunny',
        'chocolate', 'tootsie', 'm&ms', 'banana bread', 'lemon loaf', 'fudge', 'cookie', 'cookies', 'brownie', 'pancake', 'waffle', 'cinnamon raisin bagel',
        'muffin', 'pop', 'frozen pop', 'fruit bars', 'nutrigrain', 'granola bar', 'pecan twirl', 'donut', 'hershey kiss', 'toffee', 'angel food candy', 'syrup'
    ],
    'vegetarian': [
        'butter', 'cheese', 'egg', 'eggs', 'milk', 'yogurt', 'pita', 'pita bread', 'cereal', 'corn flakes', 'cornflakes', 'frosted flakes', 'granola', 'bar',
        'muffin', 'nutrigrain', 'fritos', 'chex mix', 'tootsie', 'oreo', 'fig newton', 'fig bar', 'fruit bar', 'biscotti', 'biscott', 'snickers', 'baby ruth',
        'ritz', 'cheetos', 'pistachios', 'multigrain', 'blue bunny', 'fruit bars', 'string beans', 'grits', 'grapes', 'mandarin', 'tangerine', 'banana bread',
        'salsa', 'hummus', 'peanut butter', 'peppers', 'onions', 'spinach', 'asparagus', 'cabbage', 'rice', 'beans', 'black beans', 'green bean', 'green beans',
        'squash', 'apple', 'peach', 'orange', 'navel orange', 'salad', 'waffle', 'pancake', 'cake', 'ice cream', 'chocolate', 'fudge', 'cookies', 'cookie',
        'brownie', 'lemon loaf', 'lemon risotto', 'avocado', 'flaxseed', 'psyllium husks', 'cinnamon', 'extra virgin olive oil', 'olive oil', 'balsamic vinegar',
        'lettuce mix', 'tomatoes', 'beets', 'carrots', 'cucumber', 'red bell pepper', 'brussel sprouts', 'almond nut thins', 'sweet potato', 'stuffing', 'oatmeal',
        'half and half', 'half and half', 'creamer', 'creamers', 'sugar', 'sweetner', 'equal', 'stevia', 'honey', 'raisins', 'gin soaked raisins','mozzarella',
        'corn', 'tortilla', 'bean',  'smart balance'
    ],
    'vegan': [
        'fries', 'onion', 'celery', 'bread', 'potatoes', 'peas', 'rice', 'beans', 'black beans', 'green bean', 'green beans', 'squash', 'apple', 'mandarin',
        'tangerine', 'peach', 'orange', 'navel orange', 'salad', 'asparagus', 'cabbage', 'spinach', 'peppers', 'onions', 'grapes', 'banana', 'trail mix',
        'popcorn', 'pistachios', 'multigrain', 'hummus', 'avocado', 'flaxseed', 'psyllium husks', 'cinnamon', 'extra virgin olive oil', 'olive oil', 'balsamic vinegar',
        'lettuce mix', 'tomatoes', 'beets', 'carrots', 'cucumber', 'red bell pepper', 'brussel sprouts', 'sweet potato', 'faro', 'raisins', 'potato', 'olives', 'pear',
        'grape', 'lemon', 'vegetable', 'vegetables', 'vegetable soup', 'vegetable broth', 'vegetable stir fry', 'vegetable curry', 'vegetable lasagna', 'vegetable fried rice',
        'peanut', 'clementine', 'clementines', 'kiwi', 'kiwis', 'fruit', 'berries', 'berry', 'fruit salad', 'fruit cup', 'fruit cocktail', 'fruit snacks',
    ],
    'breakfast': [
        'egg', 'eggs', 'omelet', 'cereal', 'corn flakes', 'cornflakes', 'frosted flakes', 'muffin', 'pancake', 'waffle', 'biscuit', 'bacon', 'sausage', 'grits',
        'breakfast', 'trail mix', 'yogurt', 'banana bread', 'granola bar', 'nutrigrain', 'toast', 'coffee', 'milk', 'juice', 'oatmeal', 'bagel', 'plain bagel',
        'std bfast','quaker', 'tater tot'
    ],
    'lunch': [
        'burritos', 'wrap', 'sub', 'sandwich', 'bowl', 'salad', 'chicken nuggets', 'chicken wrap', 'chicken salad', 'chicken and rice', 'chicken breast',
        'chicken thigh', 'chicken wing', 'chicken leg', 'chicken biscuit', 'chicken chorizo', 'omelet', 'egg salad', 'baked potato', 'salisbury steak', 'pot pie',
        'ravioli', 'taco salad', 'chipotle', 'deluxe cheeseburger macaroni', 'cheeseburger', 'roast beef', 'pizza', 'mac and cheese', 'ziti', 'baked cheetos',
        'totinos pizza', 'lean cuisine', 'brunswick stew', 'slaw', 'ranch'
    ],
    'dinner': [
        'steak', 'ribs', 'baked chicken', 'chicken and rice', 'chicken breast', 'chicken thigh', 'chicken wing', 'chicken leg', 'chicken biscuit', 'chicken chorizo',
        'chicken nuggets', 'omelet', 'egg salad', 'baked potato', 'salisbury steak', 'pot pie', 'ravioli', 'taco salad', 'chipotle', 'deluxe cheeseburger macaroni',
        'cheeseburger', 'roast beef', 'pizza', 'mac and cheese', 'ziti', 'baked cheetos', 'totinos pizza', 'lean cuisine', 'salmon', 'shrimp', 'seafood', 'lasagna',
        'vegetable lasagna', 'stuffing', 'brunswick stew', 'soup'
    ],
    'spicy': [
        'spicy', 'sriracha', 'jalapeno', 'buffalo', 'hot', 'kimchi', 'chili'
    ],
    'healthy': [
        'salad', 'spinach', 'asparagus', 'broccoli', 'kale', 'fruit', 'berries', 'berry', 'vegan', 'vegetarian', 'grilled', 'lactose free', 'skimmed', 'oatmeal',
        'avocado', 'flaxseed', 'psyllium husks', 'cinnamon', 'extra virgin olive oil', 'olive oil', 'balsamic vinegar', 'lettuce mix', 'tomatoes', 'beets',
        'carrots', 'cucumber', 'red bell pepper', 'brussel sprouts', 'sweet potato', 'faro', 'beet'
    ],
    'fast food': [
        'arby', 'mcdonald', 'wendy', 'burger king', 'subway', 'taco bell', 'chick-fil-a', 'jimmy dean', 'powerade', 'gatorade', 'fritos', 'cheetos', 'totinos',
        'lean cuisine', 'hamburger helper', 'red baron', 'outback steakhouse', 'trader joe', 'smoothie king', 'slim fast', 'ensure plus'
    ],
    'supplement': [
        'protein', 'whey', 'creatine', 'bcaa', 'amino', 'pre workout', 'post workout', 'casein', 'glutamine', 'l-carnitine', 'l-arginine', 'l-tyrosine',
        'mct oil', 'collagen', 'vitamin', 'mineral', 'fish oil', 'omega-3', 'probiotic', 'greens', 'fiber', 'electrolyte', 'meal replacement', 'oil'
    ],
    'medication': [
        'insulin', 'metformin', 'glipizide', 'glyburide', 'sitagliptin', 'linagliptin', 'canagliflozin', 'dapagliflozin', 'empagliflozin', 'liraglutide',
        'ibuprofen', 'aspirin', 'naproxen', 'acetaminophen', 'hydrochlorothiazide', 'lisinopril', 'atorvastatin', 'simvastatin', 'rosuvastatin', 'pravastatin'
    ]
}

def get_tags(logged_food, searched_food):
    tags = set()
    food_str = f"{logged_food or ''} {searched_food or ''}".lower()
    # Tag assignment
    for tag, keywords in tag_keywords.items():
        for kw in keywords:
            if kw in food_str:
                tags.add(tag)
                break
    # Special rules
    # If it's a drink, don't tag as entree/snack/dessert
    if 'drink' in tags:
        tags -= {'entree', 'snack', 'dessert'}
    # If it's a dessert, don't tag as snack
    if 'dessert' in tags:
        tags -= {'snack'}
    # If Vegetarian, remove vegan
    if 'vegetarian' in tags and 'vegan' in tags:
        tags.remove('vegan')
    # If it's vegan, it's also vegetarian
    if 'vegan' in tags:
        tags.add('vegetarian')
    # If it's meat, it's not vegan/vegetarian
    if 'meat' in tags:
        tags -= {'vegan', 'vegetarian'}
    # If it's seafood, it's not meat
    if 'seafood' in tags:
        tags -= {'meat'}
    # If it's breakfast, also consider as snack if not entree
    if 'breakfast' in tags and 'entree' not in tags:
        tags.add('snack')
    return sorted(tags)

# Process each entry
for entry in data:
    logged_food = entry.get('logged_food', '')
    searched_food = entry.get('searched_food', '')
    # Only process if there is a logged food
    if logged_food or searched_food:
        entry['tags'] = get_tags(logged_food, searched_food)
        if not entry['tags']:
            print(logged_food, searched_food)
    else:
        entry['tags'] = []
        print(f"missed logs {entry}")

# Save or print the updated data
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4)

Smart Balance Smart Balance
